In [1]:
!pip install pandas

In [ ]:
import json
from datetime import datetime
from dateutil import parser
import copy
import pandas as pd

LOOKUP_TABLE = {
    "event_date": {"date": parser.parse},
    "device_category": {"device_category": str},
    "geo_country": {"country": str},
    "event_name": {"event_name": str},
    "item_name": {"item_name": str},
    "item_id": {"item_id": str},
    "screen_name": {"page_title": str},
    "firebase_screen": {"content": str},
    "firebase_event_origin": {"trigger": str},
    "content_type": {"content_type": str},
    "firebase_screen_class": {"type": str},
    "ga_session_id": {"session_id": int},
    "ga_session_number": {"ga_session_number": int},
    "skywards_tier": {"skywards_tier": str},
    "event_bundle_sequence_id": {"event_count": int},
    "device_operating_system": {"operating_system": str}
}


def flatten(root, nested_dict):
    new_dict = { f"{root}_{key}": nested_dict[key] for key in nested_dict.keys()}

    return new_dict


def extract(nested_list):
    new_dict = {elem['key']: [x for x in elem['value'].values() if x is not None][0]  for elem in nested_list}

    return new_dict


with open("Raw_Data.json", "rb") as f:
    content = json.load(f)

print(content)

records = []
reworked_events = []
for event in content:
    reworked_event = copy.deepcopy(event)
    for key in event.keys():
        try:
            # Flatten all nested dict
            if type(event[key]) == dict:
                reworked_event.update(flatten(key, event[key]))
                del reworked_event[key]
        except Exception as e:
            print("Nested dictionaries have now a different structure than expected")
            raise e

        try:
            # Extract all list
            if type(event[key]) == list:
                reworked_event.update(extract(event[key]))
                del reworked_event[key]
        except Exception as e:
            print("List values have now a different format than expected")
            raise e

    reworked_events.append(reworked_event)
    tmp_dict={}
    for key in LOOKUP_TABLE.keys():
        rename_key = list(LOOKUP_TABLE[key].keys())[0]
        tmp_dict[rename_key] = reworked_event.get(key)
        if reworked_event.get(key) is not None:
            tmp_dict[rename_key] = LOOKUP_TABLE[key][rename_key](reworked_event.get(key))

    tmp_dict["downloads"] = 1 if tmp_dict["event_count"] == 1 else None
    records.append(tmp_dict)

# Uncomment below to check intermediate reworked event
# df = pd.DataFrame.from_records(reworked_events)
# df.to_csv("reworked_events.csv")

print("Results:")
print(f"Event Processed: {len(reworked_events)}")
print(f"Event Accepted: {len(records)}")



In [ ]:
df = pd.DataFrame.from_records(records)

null_values = df.isnull()
null_counts = null_values.sum()

print("Table null values:")
print(null_counts)

print("Check 'flat_table.csv' for final results")
df.to_csv("flat_table.csv")

df.head(5)